In [4]:
import numpy as np
# load the packages needed
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from src.solver import Solver
from src.model import AR, MA
from src.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
import src.data_processor as dp
import src.inference 
from src.ts_gen import ar1_gen
data = np.loadtxt("../data/SP500array.csv", delimiter=',')

In [5]:
X=np.array([data[0:100,0]])
Y=dp.get_return(X)
X = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)

In [6]:
lag = 1
sigma = 1.0
intercept = 0.1
phi = np.random.randn(1, 1)
AR_model = AR(lag=lag, phi=phi, sigma=sigma, intercept=intercept)
AR_model.params

{'intercept': 0.1, 'phi': array([[ 1.51929746]]), 'sigma': 1.0}

In [7]:
_, grads = AR_model.loss(X)
solver = Solver(AR_model, X,
                          update_rule='sgd',
                          optim_config={
                            'learning_rate': 1e-1,
                          },
                          
                          num_epochs=1000, batch_size=1,
                          print_every=10)
solver.train()

1000
the loss is %f 891.160872084
the loss is %f 830.121537038
the loss is %f 721.9494163
the loss is %f 857.772929289
the loss is %f 703.089417167
the loss is %f 565.760784033
the loss is %f 799.518095465
the loss is %f 587.912825792
the loss is %f 1056.2462628
the loss is %f 1049.61391039
the loss is %f 1042.52670301
the loss is %f 1034.91639422
the loss is %f 1026.69817065
the loss is %f 1017.76481525
the loss is %f 1007.97805109
the loss is %f 997.15525593
the loss is %f 985.048219133
the loss is %f 971.307437763
the loss is %f 955.418262452
the loss is %f 936.577221137
the loss is %f 913.425538483
the loss is %f 883.381332173
the loss is %f 840.521878272
the loss is %f 765.115958775
the loss is %f 343.569257975
the loss is %f 947.281130317
the loss is %f 926.674885385
the loss is %f 900.787957276
the loss is %f 865.944540219
the loss is %f 812.494149028
the loss is %f 694.204139209
the loss is %f 1611.03890608
the loss is %f 1259.3588443
the loss is %f 1258.56523473
the loss is %f

In [8]:
AR_model.params

{'intercept': -34.78644430473394,
 'phi': array([[-12.4692674]]),
 'sigma': 148.41166912490422}

In [9]:
AR_model.predict(X,10)

array([[ -2.89425513e+01,   3.26105967e+02,  -4.10108895e+03,
          5.11027883e+04,  -6.37249119e+05,   7.94599488e+06,
         -9.90807697e+07,   1.23546458e+09,  -1.54053382e+10,
          1.92093281e+11]])

In [23]:
from src.ts_gen import ar1_gen
import src.inference as inf

x = ar1_gen(0.5, sigma = 1, time = 200, num = 1, burnin = 2000)
X=np.array([data[0:100,0]])
Y=dp.get_return(X)

y = inf.yule_walker(x, order =1, method = "mle")
phi,sigma = inf.yule_walker(Y, order =3, method = "mle")
print phi 
print sigma



[-0.19558968 -0.04886918  0.04002632]
0.000119593894429


In [24]:

phi

array([-0.19558968, -0.04886918,  0.04002632])

In [25]:
AR_model = AR(lag=3, phi=phi, sigma=sigma, intercept=0)

In [26]:
rt=AR_model.predict(Y,10)
rt

array([[ -1.47296356e-03,  -4.76342358e-04,   5.03069047e-04,
         -1.34073966e-04,  -1.74273183e-05,   3.00966910e-05,
         -1.04014310e-05,  -1.33939402e-07,   1.73916635e-06,
         -7.49948493e-07]])

In [27]:
dp.get_price(X[0,99],rt)

array([[ 47.23232588,  47.20982712,  47.23357692,  47.22724413,
         47.22642109,  47.22784244,  47.22735121,  47.22734488,
         47.22742702,  47.2273916 ]])

In [15]:
X.shape[0]

1

In [18]:
data[100:110,0]

array([ 47.543,  46.906,  47.235,  47.235,  47.157,  46.984,  46.762,
        46.752,  46.501,  46.81 ])